<a href="https://colab.research.google.com/github/ScientistLim/ProjectAW/blob/feature%2Fdata-augmentation-Hyun/data_agumentation_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.interpolate import interp1d

# CSV 파일 불러오기 및 처리
file_name = '(0923-0930)시간-처리수전기전도도-유입압력.csv'
df = pd.read_csv(file_name)

# E열 (처리수 전기전도도), K열 (RO유입압력) 추출
df_selected = df[['Time', 'Permeate Conductivity', 'Feed Pressure']]
df_selected.columns = ['Time', 'conductivity', 'feed_pressure']

# Time 컬럼을 datetime 형태로 변환
df_selected['Time'] = pd.to_datetime(df_selected['Time'])

# 데이터 확인
print("Original Data:")
print(df_selected.head())

# 경향성을 기반으로 데이터를 16배로 늘리기
# 1. 시간 간격을 1분으로 설정하며 시간 데이터를 16배로 확장
time_min = df_selected['Time'].min()

# 데이터 간격이 1분이므로, 전체 데이터의 길이를 16배로 늘린 후 마지막 시간을 10월 1일로 맞춤
time_new = pd.date_range(start=time_min, periods=len(df_selected) * 16, freq='T')

# 2. 각 특성 데이터를 보간해서 16배로 확장
conductivity_interp = interp1d(np.arange(len(df_selected)), df_selected['conductivity'], kind='linear')
feed_pressure_interp = interp1d(np.arange(len(df_selected)), df_selected['feed_pressure'], kind='linear')

# 새로운 시간에 맞춰 데이터를 16배로 늘림
x_new = np.linspace(0, len(df_selected) - 1, len(df_selected) * 16)
conductivity_new = conductivity_interp(x_new)
feed_pressure_new = feed_pressure_interp(x_new)

# 3. feed_pressure 데이터를 점진적으로 18에 가깝게 증가시키기 위한 처리
# feed_pressure 값이 점점 증가하여 18에 수렴하도록 비율을 적용
target_value = 18
current_max = feed_pressure_new.max()
feed_pressure_new = feed_pressure_new + (target_value - current_max) * (np.linspace(0, 1, len(feed_pressure_new)))

# 확장된 데이터를 새로운 데이터프레임에 저장
df_expanded = pd.DataFrame({
    'Time': time_new,
    'conductivity': conductivity_new,
    'feed_pressure': feed_pressure_new
})

# 마지막 시간을 10월 1일로 맞추기 위한 처리
time_difference = pd.to_datetime('2024-10-01') - df_expanded['Time'].iloc[-1]
df_expanded['Time'] = df_expanded['Time'] + time_difference

# 결과 확인
print("\nExpanded Data:")
print(df_expanded.head())

# 증강된 데이터를 엑셀 파일로 저장
output_file = 'one-cycle-4months.xlsx'
df_expanded.to_excel(output_file, index=False)
print(f"Expanded data saved to {output_file}")

# 데이터 시각화
plt.figure(figsize=(10, 5))
plt.plot(df_selected['Time'], df_selected['conductivity'], label='Conductivity (Original)')
plt.plot(df_selected['Time'], df_selected['feed_pressure'], label='Feed Pressure (Original)', linestyle='--')

plt.plot(df_expanded['Time'], df_expanded['conductivity'], label='Conductivity (Expanded)', marker='o')
plt.plot(df_expanded['Time'], df_expanded['feed_pressure'], label='Feed Pressure (Expanded to 18)', marker='x')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Data Expansion with 1 Minute Interval and Time Adjusted to End on Oct 1st')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
